# Now let's do some inverse dynamics !!
While inverse kinematics is the core of biomechanics, inverse dynamics is probably its most useful information. 
It provides some insight on the internal forces that acted on the biomechanical system. 
Since the body is mostly actuated from muscles, therefore these internal forces reflect the state of the muscles at each instant.

## Load a previously created bioMod file
This first section prepares Python and load a `.bioMod` file as shown in the `2-dynamic_model_creation` or `2.5-simple_upper_limb` script with the data generated by `1-inverse_kinematics`.

In [ ]:
# So first, let's import all the required modules 
from biorbd import Biorbd  # This is the core that will do all the calculation

from copy import copy  # Allows to copy arrays
import numpy  # Numpy is a python module that helps dealing with the mathematics of matrices and vectors
numpy.set_printoptions(precision=4, suppress=True)  # Make the printing of numpy matrices prettier

from matplotlib import pyplot

In [ ]:
# Import and interpret the bioMod of the full body (created by the script `2-dynamic_model_creation` or `2.5-simple_upper_limb`)
# Please note that the dynamic model must be loaded
use_upper_limb = False  # So use the upper limb model or the full body

if use_upper_limb:
    model = Biorbd("models/SimpleUpperLimbWithInertia.bioMod")
    inverse_kinematics = numpy.load("results/inverse_kinematics_upper_limb.npy")
else:
    model = Biorbd("models/SimpleBodyWithInertia.bioMod")
    inverse_kinematics = numpy.load("results/inverse_kinematics.npy")

# Get the reconstructed kinematics
time_vector = numpy.load("results/time_vector.npy")
n_frames = inverse_kinematics.shape[1]

## Inverse Dynamics
Inverse dynamics in a nutshell is to estimate what are the forces that caused some given movement. 
Here the movement is the one created and reconstructed in the previous script, but as you may have notice, we are using the data from the reconstructed kinematics.
That means that the source of data you be some data acquisition device.

Inverse dynamics is useful to estimate the joint torques in a movement. 

In [ ]:
# First let's define a function that takes in a position (q), velocities (qdot) and accelerations (qddot)
# and spits out the net joint forces (tau) that explains this movement
def inverse_dynamics(q, qdot, qddot):
    return model.inverse_dynamics(q, qdot, qddot)

In [ ]:
# The let's test the functions using the first frame assuming no velocity nor accelerations and print the result to make sure it works as expected
q = numpy.array(inverse_kinematics[:, 0])
qdot = numpy.zeros(q.shape)
qddot = numpy.array(q.shape)
tau = inverse_dynamics(q, qdot, qddot)
if use_upper_limb:
    tau = tau[None, :]
print(f"The forces necessary to maintain the static posture with no velocity nor acceleration\n is: {tau}")

In [ ]:
# Now we have to compute the actual velocity and acceleration of the model
# We are using finite differences for that. 
# You know the drill, let's first define a function and call it once for 
# the velocities and again for the accelerations
def finite_difference(time_vector, data, window=2):
    if (window % 2 != 0):  # If window is not an even number
        raise ValueError("The window for the finite difference should be an even number")
    
    # Let's define a matrix output fill by NaN
    data_dot = numpy.ndarray(data.shape)
    data_dot[:] = numpy.nan
    
    # Now use finite difference to compute the derivative 
    # Data are expected to be derivated accross the row
    data_dot[:, int(window/2):-int(window/2)] = (data[:, window:] - data[:, :-window]) / (time_vector[window:] - time_vector[:-window])
    return data_dot


# Now we can compute the velocity and acceleration
q = inverse_kinematics
qdot = finite_difference(time_vector, q)
qddot = finite_difference(time_vector, qdot)

# First find the index of the arm x rotation in the vector of degree of freedom (this is determined by the model)
# Since the only movement in on the arm, it make sense to only print that particular dof
arm_index = model.dof_names.index("UPPER_ARM_RotX")

# Just for fun, let's plot the values for the arm
pyplot.figure()
pyplot.title("q")
pyplot.ylabel("position (rad)")
pyplot.plot(time_vector, q[arm_index, :])

pyplot.figure()
pyplot.title("velocity (rad/s)")
pyplot.ylabel("time (s)")
pyplot.plot(time_vector, qdot[arm_index, :])

pyplot.figure()
pyplot.title("qddot")
pyplot.ylabel("acceleration (rad/s/s)")
pyplot.xlabel("time (s)")
pyplot.plot(time_vector, qddot[arm_index, :])

In [ ]:
# Now that we have all the kinematics (q, qdot, qddot), we can get the net joint forces at each frame
tau = []
for i in range(n_frames):
    tau.append(inverse_dynamics(q[:, i], qdot[:, i], qddot[:, i]))
tau = numpy.concatenate(tau).T
if use_upper_limb:
    tau = tau[None, :]

# And print it for the arm
pyplot.figure()
pyplot.title("tau")
pyplot.ylabel("joint moment (N/m)")
pyplot.xlabel("time (s)")
pyplot.plot(time_vector, tau[arm_index, :])

## Mass matrix and bias (or nonlinear) effect
As bonus, we can get intermediate computation of the `tau`, namely the mass matrix ($M$) and/or the bias (or nonlinear) effects ($N$).
As reminder tau is computed as such:
\begin{align}
\tau = M(q)\ddot{q} + N(q, \dot{q})
\end{align}
So the next cell gets these intermediate values from biorbd and prints them.

For simplicity, only a single frame is shown, but you should be able to perform this over all frames pretty easily.

In [ ]:
frame = 50
Mqddot = model.mass_matrix(q[:, frame]) @ qddot[:, frame]
# The following line showcase how to access non-yet-interfaced features in biorbd-python
# Accessing "internal" allows to use features from C++ that are not yet available in Python
N = model.internal.NonLinearEffect(q[:, frame], qdot[:, frame]).to_array()
print(f"At the frame {frame}")
print(f"The M(q)*qddot part is: {Mqddot}")
print(f"The N(q,qdot) part is: {N}")
print(f"Obviously their sum ({Mqddot + N}) is the same as tau at the same frame {tau[:, frame]}")

In [ ]:
# Let's finally save the results so it can be used in the static optimisation script
import os
if not os.path.isdir("results"):
    os.mkdir("results")

if use_upper_limb:
    numpy.save("results/inverse_dynamics_upper_body.npy", (q, qdot, qddot, tau))
else:
    numpy.save("results/inverse_dynamics.npy", (q, qdot, qddot, tau))